In [30]:
import json
import re
import pandas as pd
from datetime import datetime
import locale

In [31]:
# Données JSON
data = pd.read_csv('2.5_complete_olympics_athlete_extracted_split_date_clean_isolate_site.csv')


#Fonction pour compter les compétitions
def count_competitions(text):
    if isinstance(text, str):  # Ensure text is a string
        # Updated pattern to allow single or double digit hours
        pattern = re.compile(r'(\d+ (?:juil\.|août)\d{1,2}:\d{2}.*?Terminé\(e\))', re.DOTALL)
        matches = pattern.findall(text)
        return len(matches)
    return []  # Return an empty list if text is not a string


# Appliquer la fonction à chaque ligne du DataFrame
data['competition_count'] = data['competition'].apply(count_competitions)


data.to_csv('2.6_complete_olympics_athlete_extracted_split_date_clean_isolate_site_competition_count.csv')

In [32]:


# Fonction pour extraire les premières et dernières dates
def extract_first_last_dates(competition_text):
    # Regex pour extraire les dates au format 'jour mois'
    date_strings = re.findall(r'(\d{1,2}) (juil\.|août)', competition_text)
    
    dates = []
    for day_str, month_str in date_strings:
        month_str = month_str.replace('juil.', 'Jul').replace('août', 'Aug')  # Remplacer pour le format anglais
        try:
            # Ajouter une année fictive (2024) pour le calcul
            date_obj = datetime.strptime(f"{day_str} {month_str} 2024", '%d %b %Y')
            dates.append(date_obj)
        except ValueError as e:
            print(f"Erreur de conversion pour la date: {day_str} {month_str}. Détails: {e}")

    # Identifier la première et la dernière date
    if dates:
        first_date = min(dates)
        last_date = max(dates)
        return first_date, last_date
    return None, None

# Fonction pour compter les jours uniques de compétition
def count_unique_competition_days(competition_text):
    # Extraire les dates au format 'jour mois'
    date_strings = re.findall(r'(\d{1,2}) (juil\.|août)', competition_text)
    
    # Utiliser un set pour stocker les jours uniques
    unique_days = set(date_strings)
    
    # Retourner le nombre de jours uniques
    return len(unique_days)

# Appliquer les fonctions pour chaque ligne du DataFrame
data['first_date'], data['last_date'] = zip(*data['competition'].apply(extract_first_last_dates))
data['unique_competition_days'] = data['competition'].apply(count_unique_competition_days)

# Afficher le DataFrame avec les nouvelles colonnes
print(data)

# Enregistrer le DataFrame modifié dans un fichier CSV
data.to_csv('2.7_complete_olympics_athlete_extracted_split_date_clean_isolate_site_competition_count_days_count.csv', index=False)


      web-scraper-order                        web-scraper-start-url  \
0          1727352275-1  https://olympics.com/fr/paris-2024/athletes   
1          1727352277-2  https://olympics.com/fr/paris-2024/athletes   
2          1727352279-3  https://olympics.com/fr/paris-2024/athletes   
3          1727352281-4  https://olympics.com/fr/paris-2024/athletes   
4          1727352283-5  https://olympics.com/fr/paris-2024/athletes   
...                 ...                                          ...   
11089  1727388774-11090  https://olympics.com/fr/paris-2024/athletes   
11090  1727388777-11091  https://olympics.com/fr/paris-2024/athletes   
11091  1727388780-11092  https://olympics.com/fr/paris-2024/athletes   
11092  1727388782-11093  https://olympics.com/fr/paris-2024/athletes   
11093  1727388785-11094  https://olympics.com/fr/paris-2024/athletes   

                        nom                   lien_vers_biographie  \
0                       671                                671 67

In [33]:
# Fonction pour compter les médailles
def count_medals(medals_text):
    # Regex pour extraire les médailles
    gold_count = len(re.findall(r'\bor\b', medals_text, re.IGNORECASE))
    silver_count = len(re.findall(r'2Argent', medals_text, re.IGNORECASE))
    bronze_count = len(re.findall(r'\bbronze\b', medals_text, re.IGNORECASE))
    
    return gold_count, silver_count, bronze_count

# Appliquer la fonction pour créer des colonnes pour le nombre de médailles
data[['nombre_or', 'nombre_argent', 'nombre_bronze']] = data['médailles'].apply(lambda x: pd.Series(count_medals(x)))

# Afficher le DataFrame avec le comptage des médailles
print(data[['médailles', 'nombre_or', 'nombre_argent', 'nombre_bronze']])

data.to_csv('2.8_complete_olympics_athlete_extracted_split_date_clean_isolate_site_competition_count_count_days_medals.csv')


                                               médailles  nombre_or  \
0      [{"médailles":"DisciplineÉpreuvePlaceMédaille ...          0   
1      [{"médailles":"DisciplineÉpreuvePlaceMédaille ...          0   
2      [{"médailles":"DisciplineÉpreuvePlaceMédaille ...          0   
3      [{"médailles":"DisciplineÉpreuvePlaceMédaille ...          0   
4      [{"médailles":"DisciplineÉpreuvePlaceMédaille ...          0   
...                                                  ...        ...   
11089  [{"médailles":"DisciplineÉpreuvePlaceMédaille ...          0   
11090  [{"médailles":"DisciplineÉpreuvePlaceMédaille ...          0   
11091  [{"médailles":"DisciplineÉpreuvePlaceMédaille ...          0   
11092  [{"médailles":"DisciplineÉpreuvePlaceMédaille ...          1   
11093  [{"médailles":"DisciplineÉpreuvePlaceMédaille ...          0   

       nombre_argent  nombre_bronze  
0                  0              1  
1                  1              0  
2                  1             

In [27]:
data['nombre_argent'].unique()

array([0, 1, 2, 3])

In [34]:
# Fonction pour vérifier la présence des mots clés indépendamment de leur ordre ou espacement
def count_records_paris_2024(record_text):
    # Vérifier les mots-clés indépendamment de leur espacement et de leur ordre
    has_olympic_record = all(re.search(word, record_text, re.IGNORECASE) for word in ["record", "olympique", "2024", "Paris"])
    has_world_record = all(re.search(word, record_text, re.IGNORECASE) for word in ["record", "monde", "2024", "Paris"])
    
    # Retourner 1 s'il y a un record olympique ou du monde pour Paris 2024, sinon 0
    olympic_records_count = 1 if has_olympic_record else 0
    world_records_count = 1 if has_world_record else 0
    
    return olympic_records_count, world_records_count

# Appliquer la fonction à chaque entrée et créer deux nouvelles colonnes
data[['record_olympique_count', 'record_monde_count']] = data['records'].apply(lambda x: pd.Series(count_records_paris_2024(x)))

print(data.iloc[616]['records'])
print(data.iloc[614])
# Enregistrer le DataFrame au format CSV
data.to_csv('2.9_complete_olympics_athlete_extracted_split_date_clean_isolate_site_competition_count_count_days_medals_records.csv', index=False)


[{"records":"RecordTypeTempsDatePl400 m haies - hommesRecord du monde45.9403 août 2021Tokyo400 m haies - hommesRecord olympique45.9403 août 2021Tokyo"}]
web-scraper-order                                               1727353679-615
web-scraper-start-url              https://olympics.com/fr/paris-2024/athletes
nom                                                            WARREN Portious
lien_vers_biographie                                  WARREN Portious WARREN P
lien_vers_biographie-href    https://olympics.com/fr/paris-2024/athlete/por...
discipline                                                          Athlétisme
naissance                                                         02 mars 1996
competition                  [{"competition":"Heure de débutLieuDisciplineÉ...
competition-class                                                          NaN
médailles                    [{"médailles":"DisciplineÉpreuvePlaceMédaille ...
records                                                  

In [36]:
# Liste des colonnes à supprimer
columns_to_remove = [
    'competition-class',
    'web-scraper-order', 
    'web-scraper-start-url', 
    'lien_vers_biographie', 
    'competition', 
    'médailles', 
    'records'
]

# Ajouter Competition_1 à Competition_16 et Competition_1_Date à Competition_16_Date
columns_to_remove.extend([f'Competition_{i}' for i in range(1, 17)])  # Competition_1 à Competition_16
columns_to_remove.extend([f'Competition_{i}_Date' for i in range(1, 17)])  # Competition_1_Date à Competition_16_Date


# Supprimer les colonnes
data.drop(columns=columns_to_remove, inplace=True)

# Afficher le DataFrame après suppression des colonnes
print(data)

# Enregistrer le DataFrame après suppression des colonnes dans un fichier CSV
data.to_csv('paris_olympians_data.csv', index=False)

                        nom  \
0                       671   
1         ZWOLINSKA Klaudia   
2            ZWICKER Martin   
3            ZWETSLOOT Roos   
4          ZVEREV Alexander   
...                     ...   
11089   AASMUL Victor Gairy   
11090           AARTS Laura   
11091       AARONSON Paxten   
11092         AARDAHL Maren   
11093  AABERG IVERSEN Sarah   

                               lien_vers_biographie-href          discipline  \
0      https://olympics.com/fr/paris-2024/athlete/-67...            Breaking   
1      https://olympics.com/fr/paris-2024/athlete/kla...  Canoë-kayak slalom   
2      https://olympics.com/fr/paris-2024/athlete/mar...              Hockey   
3      https://olympics.com/fr/paris-2024/athlete/roo...          Skateboard   
4      https://olympics.com/fr/paris-2024/athlete/ale...              Tennis   
...                                                  ...                 ...   
11089  https://olympics.com/fr/paris-2024/athlete/vic...  Canoë-kay